In [114]:
import pandas as pd
import numpy as np
from math import inf
from sklearn.model_selection import train_test_split
from sklearn.svm import NuSVR
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from joblib import dump #more efficient than pickle on objects that carry large numpy arrays internally


In [127]:
#df = pd.read_csv('../data/mnist/all_4hp_rusty.csv')
df = pd.read_csv('../data/mnist/all_6hp_rusty.csv')
#df = pd.read_csv('../data/mlpf/delphes_trainings_processed.csv')
df = df.iloc[:300]
df.head()

,filters,strides,max_pool,1st_dense,lr,momentum,acc_0,acc_1,acc_2,acc_3,...,loss_2,loss_3,loss_4,loss_5,loss_6,loss_7,loss_8,loss_9,loss_10,loss_11
0,128,2,3,40,0.040127,0.770056,0.914000,0.950429,0.964143,0.970357,...,0.119623,0.099236,0.101044,0.091971,0.092724,0.102770,0.089771,0.107738,0.109722,0.094621
1,128,2,5,180,0.005305,0.812338,0.946714,0.966571,0.971571,0.972071,...,0.091677,0.083224,0.068555,0.071972,0.063601,0.062818,0.061953,0.051991,0.058876,0.054147
2,32,2,5,20,0.008224,0.437781,0.920143,0.943643,0.951000,0.959000,...,0.155607,0.131733,0.121747,0.110052,0.106496,0.100451,0.094322,0.091635,0.089853,0.090477
3,128,2,5,140,0.008167,0.167884,0.933071,0.943500,0.960071,0.963714,...,0.134987,0.122413,0.106534,0.115145,0.094015,0.095907,0.097210,0.087505,0.085008,0.078288
4,16,5,3,100,0.000015,0.943157,0.720071,0.810571,0.846643,0.867857,...,0.643881,0.538333,0.473937,0.429792,0.397526,0.372900,0.353258,0.336798,0.323137,0.311195


In [116]:
num_epochs = 12
known_curve = 0.25
min_hp_idx = 0
min_curve_idx = 18
max_hp_idx =  5

hps = df[df.columns[min_hp_idx:max_hp_idx+1]].to_numpy()

curve = df[df.columns[min_curve_idx:min_curve_idx+int(num_epochs*known_curve)]].to_numpy()
target = df[df.columns[min_curve_idx+num_epochs-2]].to_numpy()

In [117]:
#calculate finite diferences of 1st and 2nd order
def finite_difs(curve):
    difs1 = []
    for i in range(curve.shape[0]):
        difs1.append([])
        for j in range(1,curve.shape[1]):
            difs1[i].append(curve[i][j]-curve[i][j-1])
    difs2 = []
    for i in range(curve.shape[0]):
        difs2.append([])
        for j in range(1,len(difs1[0])):
            difs2[i].append(difs1[i][j]-difs1[i][j-1])
    difs1 = np.array(difs1)
    difs2 = np.array(difs2)
    return difs1, difs2

In [118]:
difs1, difs2 = finite_difs(curve)
X = np.append(np.append(np.append(hps,curve,1),difs1,1),difs2,1)
y = target

In [119]:
#scale
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()
X = x_scaler.fit_transform(X)
y = y_scaler.fit_transform(y.reshape(-1, 1))
#save the scalers so that they can be used when using the SVR in another program
dump(x_scaler,"x_scaler.joblib") 
dump(y_scaler,"y_scaler.joblib")

['y_scaler.joblib']

In [120]:
#split in train test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print(str(X_train.shape)+" "+str(X_test.shape))

(17, 77) (5, 77)


In [121]:
#instatiate and train predictor
model = NuSVR()
model.fit(X_train,y_train.ravel())

NuSVR()

In [122]:
#Save model
filename = "modelo.joblib"
dump(model, "modelo.joblib")

['modelo.joblib']

In [123]:
#MSE
y_pred = model.predict(X_test)
mean_squared_error(y_test,y_pred)

0.008111866271100814

In [124]:
#R^2
model.score(X_test,y_test)

0.880142694398797

In [126]:
cvs = cross_val_score(model, X_train, y_train.ravel(), cv = 5, scoring='r2')
cvs, cvs.mean(), cvs.std()

(array([ 0.89888597,  0.50997805, -0.30507679,  0.83996273,  0.94831025]),
 0.5784120410407508,
 0.46758372635177814)

In [112]:
#try to optimize regressor hps

from sklearn.model_selection import cross_val_score

best, C_best, Nu_best, gamma_best = -inf, -inf, -inf, -inf
for _ in range(1000):
    C = np.exp(np.random.uniform(np.log(1e-3),np.log(10.0)))
    Nu = np.random.uniform(0,1)
    gamma = "scale"
    model = NuSVR(C=C,nu=Nu,gamma=gamma)
    cvs = cross_val_score(model, X_train, y_train.ravel(), cv = 5, scoring='r2').mean()
    if best < cvs:
        best = cvs
        C_best, Nu_best, gamma_best = C, Nu, gamma
best, C_best, Nu_best, gamma_best

(0.37412535448237955, 0.7488140169871008, 0.6406796854870729, 'scale')

In [888]:
model = NuSVR(C=C_best,nu=Nu_best,gamma=gamma_best)
model.fit(X_train,y_train.ravel())

cvs = cross_val_score(model, X_train, y_train.ravel(), cv = 5, scoring='r2').mean()
cvs

0.7663090107089143

In [889]:
#model.fit(X_train,y_train.ravel())
y_pred = model.predict(X_test)
mean_squared_error(y_test,y_pred), model.score(X_test,y_test)

(0.0005593760485826635, 0.9531359618257825)